In [1]:
import pandas as pd
import numpy as np
import operator

Questions for Christy:
1.) EAB column
2.) Disease Def column: differences between values
3.) What makes a street diverse?

In [2]:
df = pd.read_csv("Trees_Public.csv")

In [3]:
#df.columns

In [4]:
def dic(c):
    d = {}
    for i in c:
        try:
            d[i]+=1
        except:
            d[i]=1
    import operator
    d = sorted(d.items(), key=operator.itemgetter(1),reverse = True)
    return d
def dic2(c):
    a = {}
    for i in c:
        try:
            a[i]+=1
        except:
            a[i]=1
    import operator
    a = sorted(a.items(), key=operator.itemgetter(1),reverse = True)
    t=0
    for i in a:
        t+=i[1]
    d = []
    for i in a:
        d.append([i[0],float(i[1])/t])
    return d
def dic3(c):
    d={}
    for i in c:
        try:
            d[df[df['STREET']==i[0]]['HOOD'].iloc[0]]+=i[1]
        except:
            d[df[df['STREET']==i[0]]['HOOD'].iloc[0]]=i[1]
    for i in df['HOOD']:
        try:
            d[i]
        except:
            if i == i:
                d[i]=0
    d = sorted(d.items(), key=operator.itemgetter(1),reverse = True)
    return d

In [5]:
ash = df[df['COMMONNAME']=='Green Ash'].append(df[df['COMMONNAME']=='White Ash']) #Subset of tree dataset that only has ash trees
mt_dic = dic(ash['MT'])  #dictionary of sorted values of Maintenance columns
strs = df['STREET'].append(df['TOSTR']).append(df['FROMSTR']).append(df['ONSTR']).unique()

strs_tree_counts={}
for i in df['STREET'].unique():
    e={}
    for j in df[df['STREET']==i]['COMMONNAME']:
        #e['Ash']=0
        if j !="White Ash" and j!="Green Ash":
            try:
                e[j]+=1
            except:
                e[j]=1
        else:
            try:
                e['Ash']+=1
            except:
                e['Ash']=1
    strs_tree_counts[i]=[np.sum([values for key,values in e.items()]),e]
for key,value in strs_tree_counts.items():
    for a,b in value[1].items():
        strs_tree_counts[key][1][a] = [b,float(b)/value[0]]

bad_ash = ash[ash['MT']!='No Specific Maintenance Need']
good_ash = ash[ash['MT']=='No Specific Maintenance Need']

boulder_trees = dic(df['COMMONNAME'])

bad_ash_strs =  [i for i in bad_ash['STREET'].unique()]
bad_ash_strs_counts = sorted([[i,len(bad_ash[bad_ash['STREET'] ==i])] for i in bad_ash_strs], key = lambda num:num[1], reverse = True)
good_ash_strs = [i for i in good_ash['STREET'].unique()]
only_good_ash_strs = [i for i in good_ash['STREET'].unique() if i not in bad_ash_strs]
only_bad_ash_strs = [i for i in bad_ash['STREET'].unique() if i not in good_ash_strs]
both_ash_strs = [i for i in good_ash['STREET'].unique() if i in bad_ash['STREET'].unique()]
both_ash_strs_percent = len(both_ash_strs)/len(strs)

bad_strs_percent = len(bad_ash_strs)/len(strs)
good_strs_percent = len(good_ash_strs)/len(strs)

bad_ash_tofro_strs = bad_ash['TOSTR'].append(bad_ash['FROMSTR']).unique()


at_risk_strs = [i for i in only_good_ash_strs if i in bad_ash_tofro_strs]

at_risk_strs_counts = []
for i in at_risk_strs:
    at_risk_strs_counts.append([i,len(ash[ash['STREET']==i])])
    
at_risk_strs_sorted = sorted(at_risk_strs_counts, key=lambda num:num[1], reverse = True)

at_risk_strs_d = {}
for i in at_risk_strs_sorted:
    at_risk_strs_d[i[0]]=i[1]
    
at_risk_strs_ash_dic = {}
for i in at_risk_strs_sorted:
    at_risk_strs_ash_dic[i[0]]=strs_tree_counts[i[0]][1]['Ash']
    
at_risk_hoods=dic3(at_risk_strs_sorted)
bad_ash_hoods_counts = dic3(bad_ash_strs_counts)


strs_trees_percent_dic = {}
for i in df['COMMONNAME'].unique():
    strs_trees_percent_dic[i]=np.mean([value[1][i][1] for key,value in strs_tree_counts.items() if i in value[1]])
t=[value[1]['Ash'][1] for key,value in strs_tree_counts.items() if 'Ash'in value[1]]
strs_trees_percent_dic['Ash']=np.mean(t)
strs_trees_percent_dic_sorted = sorted(strs_trees_percent_dic.items(), key=operator.itemgetter(1),reverse = True)

strs_trees_percent_dic_2 = {}
for i in df['COMMONNAME'].unique():
    t = []
    for key,value in strs_tree_counts.items():
        if i in value[1]:
            t.append(np.mean(value[1][i][1]))
        else:
            t.append(0)
    strs_trees_percent_dic_2[i]=np.mean(t)
t=[]
for key,value in strs_tree_counts.items():
    if 'Ash' in value[1]:
        t.append(value[1]['Ash'][1])
    else:
        t.append(0)
strs_trees_percent_dic_2['Ash']=np.mean(t)
strs_trees_percent_dic_sorted_2 = sorted(strs_trees_percent_dic_2.items(), key=operator.itemgetter(1),reverse = True)

/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
help_vars = {'ash': 'filtered Data frame that only includes ash trees',
'mt_dic': 'sorted list of maintenance values for ash trees',
'boulder_trees': 'quantities of each tree in boulder',
'strs': 'list of all streets in boulder',

'bad_ash': 'data frame of all ash trees that need maintenance',
'bad_ash_strs': 'list of streets that have ash trees that need maintenance',
'only_bad_ash_strs': 'list of streets that have ash trees and all of those trees need maintenance',
'bad_ash_strs_counts': 'sorted list of streets and the amounts of bad ash trees they have',
'bad_strs_percent': 'percent of streets in Boulder that have bad trees',
'bad_ash_tofro_strs': 'list of streets that have another street touching it that have bad ash trees on it',
'bad_ash_hoods_counts': 'sorted list of hoods and the amount of bad ash trees that it has',

'good_ash': 'data frame of all ash trees that dont need maintenance',
'good_ash_strs': 'list of all streets that have good ash trees on it',
'only_good_ash_strs': 'list of streets that have ash trees and none of those trees need maintenance',
'good_strs_percent': 'percent of streets in Boulder that have good ash trees',

'both_ash_strs': 'list of streets that have some ash trees that need maintenance and some that dont',
'both_ash_strs_percent': 'percent of streets in boulder that have both good & bad ash trees',

'at_risk_strs': 'streets that have an ajoining street that has a tree that needs maintenance',
'at_risk_strs_d': 'alphabetized dictionary of streets that have no bad trees and the amount of bad trees on ajoining streets',
'at_risk_strs_sorted': 'list of streets that have no bad trees, sorted by how many bad trees are on ajoining streets',
'at_risk_strs_ash_dic': 'dictionary of streets that have neighboring streets with bad ash trees, and the amount of ash trees and the percentage of ash tree makeup of that street',
'at_risk_hoods': 'list of hoods, sorted by the amount of at risk streets they have',

'strs_trees_percent_dic': 'alphabetized dictionary of each tree, and the average percentage that the tree makes up for streets that have that tree',
'strs_trees_percent_dic_sorted': 'list of each tree, sorted by the average percentage that the tree makes up for streets that have that tree',

'strs_trees_percent_dic_2': 'alphabetized dictionary of each tree, and the average percentage of trees that that tree makes up for every street',
'strs_trees_percent_dic_sorted_2': 'list of each tree, sorted by average percentage of trees that that tree makes up for every street',

'strs_tree_counts': 'Dictionary of each street. values are counts of trees on that street, and another dictionary that has each type of tree on that street, and the count of that tree and the percentage makeup of that tree on that street'}